In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from pandas.io.json import json_normalize
import re
import json
import ast

In [47]:
#file is 20GB - preview
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
preview = pd.read_csv(r'C:\Users\philip.lawson\Desktop\kaggle\train_v2.csv',nrows=100000)

array(['{'], dtype=object)

In [4]:
f = preview.customDimensions[0]

In [5]:
json_normalize(ast.literal_eval(f))

,index,value
0,4,EMEA


In [6]:
type(preview.customDimensions[0])

str

In [7]:
preview['customDimensions'] = preview['customDimensions'].apply(ast.literal_eval)

In [8]:
json_normalize(preview['customDimensions'][7])

""


In [9]:
type(preview.customDimensions[0])

list

In [10]:
preview.customDimensions.value_counts()

[{'index': '4', 'value': 'North America'}]      43785
[]                                              20214
[{'index': '4', 'value': 'EMEA'}]               18371
[{'index': '4', 'value': 'APAC'}]               13306
[{'index': '4', 'value': 'South America'}]       2743
[{'index': '4', 'value': 'Central America'}]     1581
Name: customDimensions, dtype: int64

In [12]:
#current method ignores empty dataframes
# df_list = []
df_dict = {}
for i in np.arange(0,preview.shape[0]):
    normed = json_normalize((preview['customDimensions'][i]))
    # blank_fill = pd.DataFrame([[np.NaN,np.NaN]],columns = ['index','value'])
    # logic = np.where(normed.empty,blank_fill,normed)
    #df_list.append(normed)
    df_dict[i] = normed

In [13]:
# remove empty dataframes
df_dict_copy = df_dict.copy()
for i,j in df_dict_copy.items():
    if j.empty:
        df_dict.pop(i)

In [14]:
flattened_lookup = pd.concat(df_dict.values())
flattened_lookup.index = df_dict.keys()

In [36]:
# wrap the above in a function for other json columns
def json_column_cleaner(table_name,column_name,append = True):
    table_name[column_name] = table_name[column_name].apply(ast.literal_eval) # surplus
    df_dict = {}
    for i in np.arange(0,table_name.shape[0]):
        normed = json_normalize((table_name[column_name][i]))
        df_dict[i] = normed
    
    # remove empty dataframes
    df_dict_copy = df_dict.copy()
    for i,j in df_dict_copy.items():
        if j.empty:
            df_dict.pop(i)
    
    flattened_lookup = pd.concat(df_dict.values())
    flattened_lookup.index = df_dict.keys()
    
    # testing part
    
    if append == True:
        merged_table = table_name.merge(flattened_lookup,how='left',left_index=True,right_index=True)
        return merged_table
    else:
        return flattened_lookup

In [207]:
json_column_cleaner(preview,'customDimensions')

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,index,value
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450,4,EMEA
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307,4,North America
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613,4,North America
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851,4,EMEA
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552,4,Central America
5,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0166277907528479249,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '16',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/offer/2145"", ""campaign"": ""(...",1508196701,1,1508196701,4,North America
6,Referral,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8349655975937271469,"{""continent"": ""Europe"", ""subContinent"": ""North...","[{'hitNumber': '1', 'time': '0', 'hour': '4', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/nest-vision/dr...",1508152478,1,1508152478,4,EMEA
7,Organic Search,[],20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1332629902468998662,"{""continent"": ""Europe"", ""subContinent"": ""North...","[{'hitNumber': '1', 'time': '0', 'hour': '19',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508206208,1,1508206208,NaN,NaN
8,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",632878546807742341,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '19',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ..

In [38]:
# works for geonetwork,totals,customDimensions
json_column_cleaner(preview[:1000],'trafficSource')

ValueError: malformed node or string: <_ast.Name object at 0x0000022E634A9D30>

In [40]:
preview.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [33]:
preview.trafficSource.str.len().value_counts()

172    21434
166    17840
144    13019
182     6120
194     4938
179     4364
187     4304
165     4014
193     2076
185     1571
154     1542
380      976
197      766
186      723
191      720
183      597
189      559
173      559
200      508
215      485
196      473
190      459
176      422
201      390
192      376
370      373
402      367
136      348
203      347
188      324
       ...  
305        1
279        1
343        1
353        1
425        1
368        1
385        1
321        1
159        1
445        1
282        1
364        1
252        1
267        1
339        1
389        1
422        1
274        1
454        1
465        1
337        1
292        1
424        1
360        1
437        1
334        1
461        1
365        1
359        1
371        1
Name: trafficSource, Length: 255, dtype: int64

In [43]:
re.findall('[a-zA-Z]+":',preview.device[0])

['browser":',
 'browserVersion":',
 'browserSize":',
 'operatingSystem":',
 'operatingSystemVersion":',
 'isMobile":',
 'mobileDeviceBranding":',
 'mobileDeviceModel":',
 'mobileInputSelector":',
 'mobileDeviceInfo":',
 'mobileDeviceMarketingName":',
 'flashVersion":',
 'language":',
 'screenColors":',
 'screenResolution":',
 'deviceCategory":']

In [61]:
preview.device[0:10].str.extractall('([a-zA-Z]+):"')

,,0
,match,


1. We have 6 JSON columns containing multiple other columns
2. We have a visitStartTime which appears to be a UNIX timestamp and equal to visitId